# Week 10

# Build a bidirectional Recurrent Neural Network for any one application Workspace .

In [1]:
# bidirectional_rnn_imdb.py
# Simple Bidirectional RNN (LSTM) for IMDB sentiment classification
# Requirements: tensorflow (tested on TF 2.x)

import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
import numpy as np

# 1) Hyperparameters (small so the example runs quickly)
NUM_WORDS = 10000        # use top 10k words
MAXLEN = 200             # truncate/pad reviews to 200 tokens
EMBED_DIM = 64           # embedding size
RNN_UNITS = 64           # LSTM hidden units
BATCH_SIZE = 64
EPOCHS = 3               # small for demo; increase for better accuracy

# 2) Load dataset (pre-tokenized integer sequences)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=NUM_WORDS)

# 3) Inspect (optional)
print("Train samples:", len(x_train), "Test samples:", len(x_test))

# 4) Pad sequences so they are all the same length
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=MAXLEN, padding="post", truncating="post")
x_test  = preprocessing.sequence.pad_sequences(x_test,  maxlen=MAXLEN, padding="post", truncating="post")

# 5) Build the model
# Explanation:
# - Embedding maps token IDs -> dense vectors
# - Bidirectional wraps an LSTM so the sequence is read left->right and right->left
# - We use a small Dense head for binary classification
model = models.Sequential([
    layers.Input(shape=(MAXLEN,), dtype="int32"),
    layers.Embedding(input_dim=NUM_WORDS, output_dim=EMBED_DIM, input_length=MAXLEN),
    # You can swap the LSTM for SimpleRNN or GRU if you want to demonstrate vanilla RNN or GRU.
    layers.Bidirectional(layers.LSTM(RNN_UNITS, return_sequences=False)),
    layers.Dropout(0.5),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # binary output
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# 6) Train (small number of epochs for demonstration)
history = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=2
)

# 7) Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# 8) Quick prediction example
sample_idx = 0
sample_input = np.expand_dims(x_test[sample_idx], axis=0)
pred = model.predict(sample_input)[0,0]
print(f"\nExample review label={y_test[sample_idx]} predicted_prob={pred:.4f}")


Train samples: 25000 Test samples: 25000


C:\Users\Sujit\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 710,209 (2.71 MB)

 Trainable params: 710,209 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
313/313 - 50s - 161ms/step - accuracy: 0.7574 - loss: 0.4851 - val_accuracy: 0.8620 - val_loss: 0.3365
Epoch 2/3
313/313 - 49s - 156ms/step - accuracy: 0.8852 - loss: 0.2928 - val_accuracy: 0.8452 - val_loss: 0.3807
Epoch 3/3
313/313 - 49s - 156ms/step - accuracy: 0.9143 - loss: 0.2309 - val_accuracy: 0.8646 - val_loss: 0.3663
782/782 - 23s - 29ms/step - accuracy: 0.8458 - loss: 0.4052

Test accuracy: 0.8458, Test loss: 0.4052
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step

Example review label=0 predicted_prob=0.0606
